<a href="https://colab.research.google.com/github/zunayedology/multimodal-sensor-fusion-ppke/blob/main/Fusion03_start_with_this_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
"""
story:
let's assume we have a LIDAR
https://www.nuscenes.org/data-annotation
https://level5.lyft.com/dataset/#data-format

WE have a lidar and a detector which should detect pedestrian/traffic signs/and free space
the detector was built by someone else and we do not know anything about its quality, but we want to (and have to) use this in practice.

We want to make the best out of it using repeated measurements.

It is imporant that we asusme that these measurements are completely independent form each other...which is not ture, but we will deal with this in practice later.

"""

import numpy as np

In [2]:
"""
class Model():
  #this is a  simple probabilistic model representing our sensor
  def __init__ (self):
    #constructor

  def GenerateState(self):
    #this function generates a random hidden state
    #has no return value

  def GetState(self):
    #this is a function we can not use inn practice, it reveals the hidden state. We will use this to check our result.
    #returns a number, the index of the hidden state (0,1,2)

  def Measure(self):
    #this funciton makes a measuerement with a previously set hidden state
    #returns the index of the measured state (0,1,2)

  def MeasureKnownState(self,X):
    #this function makes a measurement with a known hidden state
    #the hidden state is the imnput aprametre of the function (0,1,2)
    #returns the index of the measured state (0,1,2)
"""

! wget https://users.itk.ppke.hu/~horan/LIDARModel.py
import LIDARModel as Model1

! wget https://users.itk.ppke.hu/~horan/LIDARModel2.py
import LIDARModel2 as Model2

--2025-09-30 06:53:11--  https://users.itk.ppke.hu/~horan/LIDARModel.py
Resolving users.itk.ppke.hu (users.itk.ppke.hu)... 193.225.109.33
Connecting to users.itk.ppke.hu (users.itk.ppke.hu)|193.225.109.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1095 (1.1K) [text/x-python]
Saving to: ‘LIDARModel.py’

LIDARModel.py       100%[===================>]   1.07K  --.-KB/s    in 0s      

2025-09-30 06:53:12 (425 MB/s) - ‘LIDARModel.py’ saved [1095/1095]

--2025-09-30 06:53:12--  https://users.itk.ppke.hu/~horan/LIDARModel2.py
Resolving users.itk.ppke.hu (users.itk.ppke.hu)... 193.225.109.33
Connecting to users.itk.ppke.hu (users.itk.ppke.hu)|193.225.109.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1101 (1.1K) [text/x-python]
Saving to: ‘LIDARModel2.py’

LIDARModel2.py      100%[===================>]   1.08K  --.-KB/s    in 0s      

2025-09-30 06:53:13 (451 MB/s) - ‘LIDARModel2.py’ saved [1101/1101]



In [3]:
# Calibration of sensor 1
M1 = Model1.Model()
# M1.State = 1

repeat = 5000
RelFreqMatrix = np.zeros((3,3))
for state in range(3):
    for i in range(repeat):
        #RelativeFrequencyMatrix[ROW, COL]
        #RelativeFrequencyMatrix[STATE, OBSERVATION]
        RelFreqMatrix[state,M1.MeasureKnownState(state)] += 1

print("Relative Frequency Matrix 1:")
print(RelFreqMatrix)
print()

#columnwise summation by axis 0
ObsProb1 = np.sum(RelFreqMatrix,0)/np.sum(RelFreqMatrix)
print("Observation Probability 1:")
print(ObsProb1)
print()

ObsMatrix1 = RelFreqMatrix/np.sum(RelFreqMatrix,1)
print("Observation Matrix 1:")
print(ObsMatrix1)
print()

Relative Frequency Matrix 1:
[[2475. 1020. 1505.]
 [1435. 3029.  536.]
 [ 498.  496. 4006.]]

Observation Probability 1:
[0.29386667 0.303      0.40313333]

Observation Matrix 1:
[[0.495  0.204  0.301 ]
 [0.287  0.6058 0.1072]
 [0.0996 0.0992 0.8012]]



In [4]:
# Calibration of sensor 2
M2 = Model2.Model()
# M2.State = 1

repeat = 5000
RelFreqMatrix = np.zeros((3,3))
for state in range(3):
    for i in range(repeat):
        #RelativeFrequencyMatrix[ROW, COL]
        #RelativeFrequencyMatrix[STATE, OBSERVATION]
        RelFreqMatrix[state,M2.MeasureKnownState(state)] += 1

print("Relative Frequency Matrix 2:")
print(RelFreqMatrix)
print()

ObsProb2 = np.sum(RelFreqMatrix,0)/np.sum(RelFreqMatrix)
print("Observation Probability 2:")
print(ObsProb2, end="\n\n")

ObsMatrix2 = RelFreqMatrix/np.sum(RelFreqMatrix,1)
print("Observation Matrix 2:")
print(ObsMatrix2, end="\n\n")

Relative Frequency Matrix 2:
[[   0. 4494.  506.]
 [1530. 3008.  462.]
 [ 507.  482. 4011.]]

Observation Probability 2:
[0.1358     0.53226667 0.33193333]

Observation Matrix 2:
[[0.     0.8988 0.1012]
 [0.306  0.6016 0.0924]
 [0.1014 0.0964 0.8022]]



In [5]:
# Entropy calculation

def Entropy(p):
  p = p.flatten()
  H = 0
  for prob in p:
    if prob != 0:
      H += -prob * np.log2(prob) # base2 log

  return H

matr = np.array([[0.35, 0.15],
                 [0.1, 0.4]])
print(Entropy(matr))

1.8016094970590273
